In [1]:
import numpy as np
import cv2

## Common CV methods

In [2]:
def display_image(img_read,title='img_display'):
    cv2.imshow(title,img_read)
    cv2.waitKey(0)    

In [3]:
def read_img(img,chn=3):
    img_read = cv2.imread(img,chn)
    display_image(img_read)
    return img_read

In [4]:
def save_img(img,img_path='img.jpg'):
    cv2.imwrite(img_path,img)

In [5]:
input_img = read_img('assignment_3/Car.png',0)

## Canny Edge detecter step by step
## Step 1 Smoothening of Image with Gaussian filter

In [52]:
def gaussian_smooth(grey_img):
    # Img Gaussian blur/smooth (Low Pass Filter)
    G_Blur_img = cv2.GaussianBlur(grey_img,(3,3), 2)
    return G_Blur_img

In [53]:
G_smooth_img = gaussian_smooth(input_img)
display_image(G_smooth_img,'Gaussian_smooth_img')
save_img(G_smooth_img,'assignment_3/result/gaussian_smooth_img.jpg')

## Step 2 Sobel Filter

In [21]:
def sobelFilter(G_Blur_img):
    # Finding Img gradient x and y
    vert_filter = np.array([[-1,0,1], [-2,0,2], [-1,0,1]]) # Sobel vertical filter
    horz_filter = np.array([[-1,-2,-1], [0,0,0], [1,2,1]]) # Sobel Horizontal filter, transpose of vertical filter
    sobel_grad_x = cv2.filter2D(G_Blur_img, -1, horz_filter)
    sobel_grad_y = cv2.filter2D(G_Blur_img, -1, vert_filter)
    # Magnitude of gradient
    abs_grad_x = cv2.convertScaleAbs(sobel_grad_x) # gradient of img in x direction
    abs_grad_y = cv2.convertScaleAbs(sobel_grad_y) # gradient of img in y direction
    grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0) # Blends images, magnitude - sobel gradient magnitude
    # Direction of the gradient
    deg = np.rad2deg(np.arctan2(abs_grad_y,abs_grad_x))
    return grad,deg

In [54]:
sobel_edges,sobel_angles = sobelFilter(G_smooth_img)
display_image(sobel_edges,'sobel_edges')
save_img(sobel_edges,'assignment_3/result/sobel_edges.jpg')

## Step 3 Non-Maximun Suppression (NMS)

In [12]:
def non_max_suppression(sobel_filt_img, angle):
    suppressed_edges = np.zeros(sobel_filt_img.shape) # create unit matrix of matrix_dimension = (r,c) and dtype=np.float32
    width,height = sobel_filt_img.shape
    # loop through each pixel location and find max neighbour pixel intesity
    for i in range(1,width-1):
        for j in range(1,height-1):
            #angle 0/180 horizontal neigbour pixel
            if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                max_neighbour_intensity = max(sobel_filt_img[i, j+1], sobel_filt_img[i, j-1])
            #angle 45, diagonal neigbour pixel from top left to bottom right
            elif (22.5 <= angle[i,j] < 67.5):
                max_neighbour_intensity = max(sobel_filt_img[i+1, j+1], sobel_filt_img[i-1, j-1])
            #angle 90 vertical neigbour pixel
            elif (67.5 <= angle[i,j] < 112.5):
                max_neighbour_intensity = max(sobel_filt_img[i+1, j], sobel_filt_img[i-1, j])
            #angle 135 (112.5 <= angle[i,j] < 157.5), diagonal neigbour pixel Top right to bottom left
            else :
                max_neighbour_intensity = max(sobel_filt_img[i-1, j+1], sobel_filt_img[i+1, j-1])
            
            # pixel intensity at location (i,j) or its neighbours is greater
            if (sobel_filt_img[i,j] >= max_neighbour_intensity):
                suppressed_edges[i,j] = sobel_filt_img[i,j]
    return suppressed_edges

In [69]:
nms_edges = non_max_suppression(sobel_edges,sobel_angles)
display_image(nms_edges,'nms_img')
save_img(nms_edges,'assignment_3/result/nms_edges.jpg')

## Step 4 Double Thresholding

In [40]:
def doubleThresholding(suppressed_edges):
    # thresold value
    low = 45
    high = 120
    
    # strong and weak pixel value
    strong = 250
    weak = 60
    #result matrix
    suppressed_dim = suppressed_edges.shape
    result_matrix = np.zeros(suppressed_dim)
    # location x,y of strong and weak pixels in the image suppressed_edges
    strong_x, strong_y = np.where(suppressed_edges>=high)
    weak_x, weak_y = np.where((suppressed_edges>low) & (suppressed_edges<high))
    
    result_matrix[strong_x, strong_y] = strong
    result_matrix[weak_x, weak_y] = weak
    
    return result_matrix

In [70]:
doubleThreshold_img=doubleThresholding(nms_edges)
display_image(doubleThreshold_img,'doubleThreshold_car_edges')
save_img(doubleThreshold_img,'assignment_3/result/doubleThreshold_img.jpg')

## Step 5 Edge Tracking

In [57]:
#loop through the img and check if any neighbour of weak pixel is strong
def hysteresis_edgeTracking(img, weak, strong=255):
    height, width = img.shape  
    for i in range(1, height-1):
        for j in range(1, width-1):
            if (img[i,j] == weak): # check for strong pixels in the weak pixel neighbour
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img

In [74]:
hysteresis_img = hysteresis_edgeTracking(doubleThreshold_img,60,250)
display_image(hysteresis_img,'hysteresis_car_edges')
save_img(hysteresis_img,'assignment_3/result/hysteresis_img.jpg')